In [1]:
import cv2
import mediapipe as mp
import numpy as np
import os

def mediapipeDetectHands(image, hands_module, min_detection_confidence=0.5, display=True):
    image_height, image_width, _ = image.shape
    output_image = image.copy()

    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    results = hands_module.process(img_rgb)

    detected_hands = []

    # Set the scaling factor for augmentation
    scaling_factor = 1.5

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for landmark in hand_landmarks.landmark:
                x = int(landmark.x * image_width)
                y = int(landmark.y * image_height)
                landmarks.append((x, y))

            # Get the bounding box of the hand
            bbox = cv2.boundingRect(np.array(landmarks))

            x1, y1, w, h = bbox

            # Augment the bounding box dimensions
            x1 = int(x1 * scaling_factor)
            y1 = int(y1 * scaling_factor)
            w = int(w * scaling_factor)
            h = int(h * scaling_factor)

            # Ensure that the augmented bounding box is within the image boundaries
            x1 = max(0, x1)
            y1 = max(0, y1)
            x2 = min(image_width, x1 + w)
            y2 = min(image_height, y1 + h)

            # Draw the augmented rectangle around the hand
            cv2.rectangle(output_image, pt1=(x1, y1), pt2=(x2, y2), color=(0, 255, 0), thickness=2)

            detected_hands.append(landmarks)

            # Draw the hand landmarks on the output image
            mp.solutions.drawing_utils.draw_landmarks(
                output_image, hand_landmarks, mp.solutions.hands.HAND_CONNECTIONS,
                landmark_drawing_spec=mp.solutions.drawing_styles.get_default_hand_landmarks_style()
            )

    if display:
        cv2.imshow("Output", output_image)
    else:
        return output_image, detected_hands
# Path to the training data directory
DATA_DIR = 'behav_detector/data/train'

# Output directory to save the corrected images
OUTPUT_DIR = 'behav_detector/cropped_data2'

# Create the output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Initialize the MediaPipe Hands module
mp_hands = mp.solutions.hands
hands_module = mp_hands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Loop through each class in the training data directory
for class_name in os.listdir(DATA_DIR):
    class_dir = os.path.join(DATA_DIR, class_name)
    output_class_dir = os.path.join(OUTPUT_DIR, class_name)
    os.makedirs(output_class_dir, exist_ok=True)

    # Loop through each image in the class directory
    for img_name in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_name)
        img = cv2.imread(img_path)

        # Check if the image is read successfully
        if img is not None:
            output_image, detected_hands = mediapipeDetectHands(img, hands_module, display=False)

            # Save the corrected hand images to the output directory
            for i, landmarks in enumerate(detected_hands):
                # Construct the output file name without ".jpg" in the middle
                output_name = f'{img_name}_hand_{i}'
                output_path = os.path.join(output_class_dir, f'{output_name}.jpg')

                # Draw rectangle and save the hand region
                x1, y1, w, h = cv2.boundingRect(np.array(landmarks))

                # Print debug information
                print(f"Image: {img_name}, Hand {i} - Coordinates: x={x1}, y={y1}, w={w}, h={h}")

                # Check if the image region is empty or invalid
                if h > 0 and w > 0 and y1 + h <= img.shape[0] and x1 + w <= img.shape[1]:
                    cv2.imwrite(output_path, img[y1:y1 + h, x1:x1 + w])
                else:
                    print(f"Skipping saving image {output_path} due to empty or invalid region")

# Release the resources used by the hands module
hands_module.close()

# Close OpenCV windows
cv2.destroyAllWindows()


FileNotFoundError: [WinError 3] Le chemin d’accès spécifié est introuvable: 'data/train'